In [103]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error

# LinearRegression

Observations from initial linear regression


1. The scores are more important than the trends

2. All combined is the best model

3. Only trends is worse

4. Only scores is close

5. There is no variance. (So increase features)

In [104]:
def mape(y_true,y_predict):
    return np.sum((np.abs(y_true-y_predict)/y_true)*100)/len(y_true)

In [105]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
feature_df = pd.read_csv('csv_data/first_innings_feature_expanded.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

team_df = pd.read_csv('csv_data/first_innings_team.csv')
opponent_df = pd.read_csv('csv_data/first_innings_opponent.csv')
team_batsman_df = pd.read_csv('csv_data/first_innings_batsman.csv')
opponent_bowler_df= pd.read_csv('csv_data/first_innings_opponent_bowler.csv')

#temporary correction code
# team_df['match_id']=feature_df['match_id']
# opponent_df['match_id']=feature_df['match_id']
# team_batsman_df['match_id']=feature_df['match_id']
# opponent_bowler_df['match_id']=feature_df['match_id']




In [106]:
total_df = feature_df.merge(team_df,on='match_id',how='inner',suffixes=('','_team')).\
merge(opponent_df,on='match_id',how='inner',suffixes=('','_opp')).\
merge(team_batsman_df,on='match_id',how='inner',suffixes=('','_bat')).\
merge(opponent_bowler_df,on='match_id',how='inner',suffixes=('','_bow'))



In [110]:
total_df.shape,feature_df.shape

((1129, 47), (1129, 20))

In [118]:
total_df['match_id'].nunique(),total_df.shape

(1024, (1024, 47))

In [111]:
total_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'runs_scored',
       'opponent_base', 'opponent_trend', 'opponent_trend_predict',
       'opponent_mean', 'location_base', 'location_trend',
       'location_trend_predict', 'location_mean', 'current_base',
       'current_trend', 'current_trend_predict', 'current_mean', 'is_train',
       'noise', 'country', 'win_ratio', 'effective_win_by_runs',
       'effective_win_by_wickets', 'matches_played', 'win_count',
       'country_opp', 'win_ratio_opp', 'effective_win_by_runs_opp',
       'effective_win_by_wickets_opp', 'matches_played_opp', 'win_count_opp',
       'total_runs', 'run_rate', 'average_score', 'opponent_variability',
       'player_of_the_match', 'winning_contribution', 'run_rate_effectiveness',
       'negative_rate', 'no_of_wickets', 'wickets_per_match',
       'wickets_per_run', 'no_of_wins', 'opponent_variability_bow',
       'winning_contribution_bow', 'winning_wicket_rate_contribution'],
      dtype='obje

In [112]:
#feature_df.dropna(inplace=True)
total_df.isnull().sum()

match_id                             0
match_date                           0
team                                 0
opponent                             0
location                             0
runs_scored                          0
opponent_base                        0
opponent_trend                       0
opponent_trend_predict               0
opponent_mean                        0
location_base                        0
location_trend                       0
location_trend_predict               0
location_mean                        0
current_base                         0
current_trend                        0
current_trend_predict                0
current_mean                         0
is_train                             0
noise                                0
country                              0
win_ratio                            0
effective_win_by_runs                0
effective_win_by_wickets             0
matches_played                       0
win_count                

In [113]:
total_df.dropna(inplace=True)

In [114]:
total_df['match_id'].nunique()

1024

In [202]:
# feature_columns = ['team_score',
#        'opponent_score', 'opponent_base', 'opponent_trend',
#        'opponent_trend_predict', 'opponent_mean', 'location_base',
#        'location_trend', 'location_trend_predict', 'location_mean',
#        'current_base', 'current_trend', 'current_trend_predict',
#        'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
#        'bowler_max']

feature_columns = [
        'location_mean',  'run_rate', 'average_score', 'opponent_variability','wickets_per_run', ]
       


target_column = ['runs_scored']

In [203]:
feature_df_train = total_df[total_df['is_train']==True]
feature_df_test = total_df[total_df['is_train']==False]

In [204]:
feature_df_train[feature_columns].dtypes

location_mean           float64
run_rate                float64
average_score           float64
opponent_variability    float64
wickets_per_run         float64
dtype: object

In [205]:
feature_df_train.dtypes

match_id                                     int64
match_date                          datetime64[ns]
team                                        object
opponent                                    object
location                                    object
runs_scored                                  int64
opponent_base                              float64
opponent_trend                             float64
opponent_trend_predict                     float64
opponent_mean                              float64
location_base                              float64
location_trend                             float64
location_trend_predict                     float64
location_mean                              float64
current_base                               float64
current_trend                              float64
current_trend_predict                      float64
current_mean                               float64
is_train                                      bool
noise                          

In [206]:
scaler = StandardScaler()
y_scaler = StandardScaler()
x_train = scaler.fit_transform(feature_df_train[feature_columns])
y_train = feature_df_train[target_column]
y_train_sc = y_scaler.fit_transform(feature_df_train[target_column])

x_test = scaler.transform(feature_df_test[feature_columns])
y_test = feature_df_test[target_column]
y_test_sc = y_scaler.transform(feature_df_test[target_column])




# x_train = np.array(feature_df_train[feature_columns])
# y_train = np.array(feature_df_train[target_column])

# x_test = np.array(feature_df_test[feature_columns])
# y_test = np.array(feature_df_test[target_column])




In [207]:
lr = LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [208]:
y_train_predict = lr.predict(x_train)
y_test_predict = lr.predict(x_test)

In [209]:
mean_absolute_error(y_train,y_train_predict)

47.419301645403884

In [210]:
mean_absolute_error(y_test,y_test_predict)

48.44396220648048

In [211]:
mape(np.array(y_train),np.array(y_train_predict))

23.113251645975332

In [212]:
mape(np.array(y_test),np.array(y_test_predict))

24.60888150275529

In [213]:
y_test

,runs_scored
933,371
934,319
935,364
936,288
937,298
...,...
1120,294
1121,231
1122,302
1127,374


In [214]:
compare_df = pd.DataFrame()
compare_df['actual']=y_test['runs_scored']
compare_df['predicted']=y_test_predict.reshape(-1)

In [215]:
compare_df['percentage_error']=abs(compare_df['actual']-compare_df['predicted'])/compare_df['actual']

In [216]:
compare_df

,actual,predicted,percentage_error
933,371,256.740564,0.307977
934,319,311.114548,0.024719
935,364,307.254305,0.155895
936,288,270.262069,0.061590
937,298,284.722651,0.044555
...,...,...,...
1120,294,247.444750,0.158351
1121,231,260.704778,0.128592
1122,302,259.099443,0.142055
1127,374,289.810637,0.225105


In [162]:
compare_df['is_less']=compare_df['actual']>compare_df['predicted']

In [163]:
compare_df[compare_df['is_less']==True].shape,compare_df.shape

((94, 4), (177, 4))

In [164]:
result_df =pd.concat([feature_df_test,compare_df],axis=1)

In [165]:
result_df[result_df['percentage_error']<0.15]

,match_id,match_date,team,opponent,location,runs_scored,opponent_base,opponent_trend,opponent_trend_predict,opponent_mean,...,wickets_per_match,wickets_per_run,no_of_wins,opponent_variability_bow,winning_contribution_bow,winning_wicket_rate_contribution,actual,predicted,percentage_error,is_less
934,1153841,2019-01-05,New Zealand,Sri Lanka,Mount Maunganui,319,283.7,9.1,338.3,311.00,...,1.750000,0.040660,1.75,3.50,0.047297,0.284091,319,317.129601,0.005863,True
935,1153842,2019-01-08,New Zealand,Sri Lanka,Nelson,364,258.5,16.7,358.7,308.60,...,1.645833,0.041207,2.25,3.50,0.060811,0.269886,364,315.083618,0.134386,True
936,1144997,2019-01-12,Australia,India,Sydney,288,259.1,6.1,295.7,277.40,...,1.763622,0.042136,8.50,4.25,0.070675,0.210978,288,252.042947,0.124851,True
937,1144998,2019-01-15,Australia,India,Adelaide,298,305.9,-5.7,271.7,288.80,...,1.763622,0.036870,7.75,4.00,0.063291,0.204360,298,265.592598,0.108750,True
938,1144999,2019-01-18,Australia,India,Melbourne,230,297.6,-2.6,282.0,289.80,...,1.328205,0.035812,8.75,4.75,0.058371,0.168256,230,249.790758,0.086047,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,1187684,2020-02-11,India,New Zealand,Mount Maunganui,296,171.0,20.6,294.6,232.80,...,1.564015,0.036011,6.25,6.75,0.063953,0.269830,296,297.499837,0.005067,False
1108,1213871,2020-02-22,West Indies,Sri Lanka,Colombo,289,161.0,18.3,252.5,206.75,...,1.188889,0.025742,2.75,5.00,0.061728,0.201684,289,271.839098,0.059380,True
1114,1185320,2020-03-04,Australia,South Africa,Bloemfontein,271,209.8,2.0,221.8,215.80,...,1.641369,0.037996,2.50,3.75,0.032362,0.179275,271,282.492827,0.042409,False
1116,1183537,2020-03-13,Australia,New Zealand,Sydney,258,275.0,-7.0,233.0,254.00,...,1.577904,0.038528,6.75,8.25,0.087209,0.192965,258,288.862083,0.119620,False


In [166]:
for country in result_df['team'].unique():
    predicted_series=result_df[result_df['team']==country]['predicted']
    actual_series=result_df[result_df['team']==country]['actual']

    print(country,mape(np.array(actual_series),np.array(predicted_series)))

New Zealand 20.309070047909724
Australia 11.439872313526184
South Africa 39.08748193024697
Pakistan 30.378321860407237
India 29.98145334528683
United Arab Emirates 12.984281548121638
West Indies 14.985897141434686
Bangladesh 12.936016012204743
Sri Lanka 21.5466637031942
England 28.937236019011998
Ireland 17.064778044826106
Afghanistan 13.892645093395103
Zimbabwe 14.8539380902467
Scotland 1.9020442328688467
Papua New Guinea 14.82377211174067
United States of America 114.87245444391336
Namibia 18.214530657442467
Nepal 16.637484303872025
Oman 20.538733333554312


In [167]:
for country in result_df['team'].unique():
    std = result_df[result_df['team']==country]['actual'].std()
    print(country,std)

New Zealand 76.28031013496022
Australia 45.05123578941921
South Africa 68.80186748698736
Pakistan 77.5970855915845
India 80.57913730253665
United Arab Emirates 35.2514775104061
West Indies 61.15109192774929
Bangladesh 44.75468850139774
Sri Lanka 70.44011303092525
England 85.17195217856931
Ireland 56.2200714031157
Afghanistan 38.82009788756334
Zimbabwe 45.254833995939045
Scotland 13.435028842544403
Papua New Guinea 9.192388155425117
United States of America 95.14620328736193
Namibia 45.254833995939045
Nepal nan
Oman nan


# lasso

In [168]:
from sklearn import linear_model

In [169]:
las = linear_model.Lasso(alpha=0.3)
las.fit(x_train,y_train_sc)

Lasso(alpha=0.3)

In [170]:
y_test_predict_ls = las.predict(x_test)
y_train_predict_ls = las.predict(x_train)

In [171]:
mape(np.array(y_train_sc).reshape(-1),np.array(y_train_predict_ls))

4.604486422668232

In [172]:
mape(np.array(y_test_sc).reshape(-1),np.array(y_test_predict_ls))

3.9548022598870056

In [173]:
y_test_predict_original=y_scaler.inverse_transform(y_test_predict_ls)
#y_test_original = feature_df_test[target_column]

In [174]:
mape(np.array(y_test).reshape(-1),np.array(y_test_predict_original))

27.818898797291652

In [175]:
y_test_predict_ls

array([4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
       4.19446715e-18, 4.19446715e-18, 4.19446715e-18, 4.19446715e-18,
      

In [176]:
np.array(y_test).reshape(-1)

array([371, 319, 364, 288, 298, 230, 266, 203, 192, 324, 164, 243, 254,
       161, 240,  92, 149, 252, 216, 240, 225, 232, 226, 139, 330, 360,
       289, 202, 241, 247, 161, 418, 205, 161, 236, 113, 231, 231, 250,
       256, 251, 223, 313, 216, 358, 272, 189, 187, 174, 280, 284, 266,
       277, 327, 198, 381,  63, 261, 147, 373, 327, 247, 358, 292, 340,
       351, 210, 305, 311, 105, 207, 136, 330, 348, 227, 244, 288, 318,
       172, 386, 352, 307, 212, 334, 321, 397, 241, 381, 232, 224, 291,
       308, 262, 285, 237, 268, 203, 243, 227, 337, 338, 254, 314, 177,
       305, 217, 311, 242, 315, 264, 325, 269, 190, 239, 223, 314, 238,
       294, 223, 242, 206, 180, 177, 219, 260, 305, 297, 164, 247, 146,
       191, 194, 194, 247, 249, 202, 284, 282, 327, 245, 287, 387, 315,
       170, 222, 180, 324, 237, 255, 340, 286, 259, 115, 149, 258, 347,
       213, 273, 249, 256, 276, 296,  35, 289, 345, 291, 307, 321, 322,
       271, 254, 258, 294, 231, 302, 374, 389])

# XGBOOST

In [177]:
import xgboost as xg

In [178]:
train_dmatrix = xg.DMatrix(data = x_train, label = y_train) 
test_dmatrix = xg.DMatrix(data = x_test, label = y_test) 

In [179]:
param = {"booster":"gblinear", "objective":"reg:linear"} 

In [180]:
xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 10) 


[17:10:53] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:53] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.


In [184]:
y_test_predict_xg = xgb_r.predict(test_dmatrix)
y_train_predict_xg = xgb_r.predict(train_dmatrix)

In [185]:
# y_train_predict_dt = xgb_r.predict(x_train)
# y_test_predict_dt = xgb_r.predict(x_test)

In [186]:
mean_absolute_error(np.array(y_test),np.array(y_test_predict_xg))

47.379097890045685

In [187]:
mape(np.array(y_test).reshape(-1),y_test_predict_xg)

24.069187019176173

In [188]:
mape(np.array(y_train).reshape(-1),y_train_predict_xg)

22.232527241609365

In [189]:
#y_test_predict_xg

# Randomforest

In [190]:
from sklearn.ensemble import RandomForestRegressor

In [191]:
rf = RandomForestRegressor(max_depth=8,n_estimators=50,criterion='mae')

In [192]:
rf.fit(x_train,y_train)

<ipython-input-192-895403b03771>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(x_train,y_train)


RandomForestRegressor(criterion='mae', max_depth=8, n_estimators=50)

In [193]:
y_train_predict_rf=rf.predict(x_train)
y_test_predict_rf= rf.predict(x_test)

In [194]:
mape(np.array(y_test).reshape(-1),y_test_predict_rf)

24.95595654717835

In [195]:
mape(np.array(y_train).reshape(-1),y_train_predict_rf)

15.30899016596822

# statsmodel

In [196]:
import statsmodels.api as sm

In [197]:
model = sm.OLS(y_train, sm.add_constant(x_train)).fit()

In [198]:
y_test_predicted_sm=model.predict(sm.add_constant(x_test))

In [199]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            runs_scored   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     6.762
Date:                Thu, 10 Dec 2020   Prob (F-statistic):           1.56e-21
Time:                        17:11:28   Log-Likelihood:                -4653.3
No. Observations:                 847   AIC:                             9361.
Df Residuals:                     820   BIC:                             9489.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        252.9787      2.055    123.099      0.000     248.945     257.013
x1             6.3890      2.342      2.728      0.007       1.791      10.987
x2            -3.3609      4.907     -0.685      0.494     -12.993       6.271
x3             2.3739      4.264      0.557      0.578      -5.995      10.743
x4            -2.5693      4.658     -0.552      0.581     -11.712       6.574
x5            -6.8558     10.356     -0.662      0.508     -27.182      13.471
x6            15.8630     14.135      1.122      0.262     -11.882      43.608
x7             1.1785      4.802      0.245      0.806      -8.246      10.603
x8            -7.7447      4.174     -1.855      0.064     -15.938       0.449
x9            -4.2768      4.704     -0.909      0.364     -13.510       4.956
x10          -18.5680      9.636     -1.927      0.054     -37.482       0.346
x11           15.0751     14.479      1.041      0.298     -13.345      43.495
x12           10.7572      5.689      1.891      0.059      -0.409      21.923
x13            7.2037      3.280      2.196      0.028       0.765      13.642
x14            6.6175      3.067      2.158      0.031       0.597      12.638
x15           -8.9587      3.566     -2.512      0.012     -15.958      -1.960
x16           -0.7336      4.216     -0.174      0.862      -9.009       7.542
x17           -2.0640      3.824     -0.540      0.590      -9.571       5.443
x18            1.0788      2.778      0.388      0.698      -4.375       6.532
x19          -10.1653      3.196     -3.181      0.002     -16.439      -3.892
x20            9.4236      6.745      1.397      0.163      -3.817      22.664
x21           -8.8142      4.573     -1.927      0.054     -17.791       0.163
x22           11.3541      4.385      2.589      0.010       2.747      19.961
x23          -12.1922      7.261     -1.679      0.094     -26.445       2.060
x24            6.4297      3.438      1.870      0.062      -0.319      13.178
x25            0.8819      4.072      0.217      0.829      -7.111       8.874
x26           -2.1696      2.990     -0.726      0.468      -8.039       3.700
==============================================================================
Omnibus:                       11.004   Durbin-Watson:                   1.840
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               12.444
Skew:                          -0.202   Prob(JB):                      0.00199
Kurtosis:                       3.436   Cond. No.                         22.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [200]:
mape(np.array(y_test).reshape(-1),y_test_predicted_sm)


23.998004935171835

In [201]:
mean_absolute_error(np.array(y_test).reshape(-1),y_test_predicted_sm)

47.36892179267138

# with PCA

In [1154]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

In [1155]:
pca_scaler = StandardScaler()

In [1156]:
pca = PCA(n_components=2)

In [1157]:
x_train_pca=pca.fit_transform(pca_scaler.fit_transform(x_train))

In [1158]:
lr_pca = LinearRegression()
lr_pca.fit(x_train_pca,y_train)

LinearRegression()

In [1159]:
y_test_predict_pca =lr_pca.predict(pca.transform(pca_scaler.transform(x_test)))

In [1160]:
mape(np.array(y_test),y_test_predict_pca)

24.732481820472458

In [1161]:
#y_test_predict_pca.shape

In [1162]:
#np.array(y_test).reshape(-1).shape

# SVM

In [1163]:
from sklearn.svm import SVR

In [1164]:
svr = SVR(C=0.5, epsilon=0.01)

In [1165]:
svr.fit(x_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR(C=0.5, epsilon=0.01)

In [1166]:
y_test_predict_svr =svr.predict(x_test)

In [1167]:
mape(np.array(y_test).reshape(-1),y_test_predict_svr)

25.484214301334358

# polynomial regression

In [1168]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures

In [1186]:
poly = PolynomialFeatures(2)

In [1187]:
x_train_poly = poly.fit_transform(x_train)

In [1188]:
lr_poly = LinearRegression()

In [1189]:
lr_poly.fit(x_train_poly,y_train)

LinearRegression()

In [1190]:
y_test_predict_poly = lr_poly.predict(poly.transform(x_test))
y_train_predict_poly = lr_poly.predict(poly.transform(x_train))

In [1191]:
mape(np.array(y_test),y_test_predict_poly)

23.44766139579654

In [1192]:
mape(np.array(y_train),y_train_predict_poly)

21.24022417431785

In [1193]:
compare_df = pd.DataFrame()
compare_df['actual']=np.array(y_test).reshape(-1)
compare_df['predict']=np.array(y_test_predict_poly)

In [1177]:
compare_df

,actual,predict
0,371,294.315819
1,319,294.215821
2,364,300.301850
3,288,270.437914
4,266,298.658391
...,...,...
133,180,204.664955
134,252,218.824205
135,325,309.958348
136,374,275.687084


In [1178]:
np.array(y_test).reshape(-1).shape

(138,)

# Comparing trend predictions

In [1067]:
feature_df.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'team_score',
       'opponent_score', 'opponent_base', 'opponent_trend',
       'opponent_trend_predict', 'opponent_mean', 'location_base',
       'location_trend', 'location_trend_predict', 'location_mean',
       'current_base', 'current_trend', 'current_trend_predict',
       'current_mean', 'batsman_mean', 'batsman_max', 'bowler_mean',
       'bowler_max', 'is_train', 'noise', 'runs_scored'],
      dtype='object')

In [1068]:
feature_df[['opponent_trend_predict','runs_scored']]

,opponent_trend_predict,runs_scored
0,124.0,293
1,263.7,249
2,10.0,262
3,274.0,333
4,255.8,279
...,...,...
780,210.6,180
781,175.1,252
782,243.2,325
783,311.8,374


In [1070]:
opponent_trend_predict = np.array(feature_df['opponent_trend_predict'])
current_trend_predict = np.array(feature_df['current_trend_predict'])
location_trend_predict = np.array(feature_df['location_trend_predict'])
runs_scored = np.array(feature_df['runs_scored'])

In [1073]:
mape(runs_scored,opponent_trend_predict),mape(runs_scored,current_trend_predict),mape(runs_scored,location_trend_predict)

(34.431899956652344, 33.545952098392334, 36.5804185718854)

In [1078]:
max_predict = np.max(np.array(feature_df[['opponent_trend_predict','current_trend_predict','location_trend_predict']]),axis=1)
min_predict = np.min(np.array(feature_df[['opponent_trend_predict','current_trend_predict','location_trend_predict']]),axis=1)
mean_predict = np.mean(np.array(feature_df[['opponent_trend_predict','current_trend_predict','location_trend_predict']]),axis=1)


In [1080]:
mape(runs_scored,max_predict),mape(runs_scored,min_predict),mape(runs_scored,mean_predict)

(40.1016063772506, 34.66377592839274, 28.793546905523176)